In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load data

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('/kaggle/input/protein-secondary-structure/2018-06-06-ss.cleaned.csv')
print(df.shape)

def seq2ngrams(seqs, n=3):
    return np.array([[seq[i:i+n] for i in range(len(seq))] for seq in seqs])

df = df.head(5001)
maxlen_seq = 500
input_seqs, target_seqs = df[['seq', 'sst3']][(df.len <= maxlen_seq) & (~df.has_nonstd_aa)].values.T
input_grams = seq2ngrams(input_seqs)
print(len(input_seqs))

In [ ]:

df.len.hist(bins=100)
print(df.shape)

In [ ]:
from collections import defaultdict
# show proportion of each amino acid in a table
AA_counts = {'ss_2018': defaultdict(lambda: 0)}
# count the types for each dataset
for (seq, nonstd) in zip(df['seq'], df['has_nonstd_aa']):
    if not nonstd:
        for aa in set(seq):
            if aa != '*':
                AA_counts['ss_2018'][aa] += seq.count(aa)

# order the amino acids by decreasing total abundance
total_aa = [sum([AA_counts[d][aa] for d in AA_counts.keys()]) for aa in AA_counts['ss_2018'].keys() ]
temp = sorted(total_aa, reverse = True)    
order = [total_aa.index(v) for v in temp]
aa_order = [list(AA_counts['ss_2018'].keys())[i] for i in order]
tbl_data = {'Amino Acid': aa_order,
            'ss_2018': [ round(AA_counts['ss_2018'][aa] / sum(AA_counts['ss_2018'].values()), 3) for aa in aa_order]}
pd.DataFrame(tbl_data)

In [ ]:
import seaborn as sns
dict_a = {'C':[],'H':[],'E':[]}
dict_f = {'C':[],'H':[],'E':[]}
dict_p = {'C':[],'H':[],'E':[]}
dict_s = {'C':[],'H':[],'E':[]}

for se,sst in zip(df['seq'],df['sst3']):
    for s,ss in zip(se,sst):
        if s == 'A':
            if ss == 'C': dict_a['C'].append(1)
            elif ss == 'H': dict_a['H'].append(1)
            elif ss == 'E': dict_a['E'].append(1)
        elif s == 'F':        
            if ss == 'C': dict_f['C'].append(1)
            elif ss == 'H': dict_f['H'].append(1)
            elif ss == 'E': dict_f['E'].append(1)
        elif s == 'P':        
            if ss == 'C': dict_p['C'].append(1)
            elif ss == 'H': dict_p['H'].append(1)
            elif ss == 'E': dict_p['E'].append(1)
        elif s == 'S':        
            if ss == 'C': dict_s['C'].append(1)
            elif ss == 'H': dict_s['H'].append(1)
            elif ss == 'E': dict_s['E'].append(1)
                
                
for k in dict_a.keys(): dict_a[k] = sum(dict_a[k])
for k in dict_f.keys(): dict_f[k] = sum(dict_f[k])
for k in dict_p.keys(): dict_p[k] = sum(dict_p[k])
for k in dict_s.keys(): dict_s[k] = sum(dict_s[k])
print('dict_a:  ',dict_a)
print('dict_f:  ',dict_f)
print('dict_p:  ',dict_p)
print('dict_s:  ',dict_s)

plt.figure(figsize=(14,4));
plt.subplot(1,2,1);
sns.barplot(x=list(dict_a.keys()),y=list(dict_a.values()),color='gray');
plt.title('Secondary Structure character counts for aminoacid A');
plt.subplot(1,2,2);
sns.barplot(x=list(dict_f.keys()),y=list(dict_f.values()),color='gray');
plt.title('Secondary Structure character counts for aminoacid F');
plt.figure(figsize=(14,4));
plt.subplot(1,2,1);
sns.barplot(x=list(dict_p.keys()),y=list(dict_p.values()),color='gray');
plt.title('Secondary Structure character counts for aminoacid P');
plt.subplot(1,2,2);
sns.barplot(x=list(dict_s.keys()),y=list(dict_s.values()),color='gray');
plt.title('Secondary Structure character counts for aminoacid S');

# Bi-LSTM to predict secondary structure

## Preprocessing

In [ ]:
def seq2ngrams(seqs, n=3):
    return np.array([[seq[i:i+n] for i in range(len(seq))] for seq in seqs])

maxlen_seq = 500
input_seqs, target_seqs = df[['seq', 'sst3']][(df.len <= maxlen_seq) & (~df.has_nonstd_aa)].values.T
input_grams = seq2ngrams(input_seqs)
print(len(input_seqs))

In [ ]:
from keras.preprocessing import text
from keras.utils.data_utils import pad_sequences 
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

tokenizer_encoder = Tokenizer()
tokenizer_encoder.fit_on_texts(input_grams)
input_data = tokenizer_encoder.texts_to_sequences(input_grams)
input_data = pad_sequences(input_data, maxlen=maxlen_seq, padding='post')

tokenizer_decoder = Tokenizer(char_level=True)
tokenizer_decoder.fit_on_texts(target_seqs)
target_data = tokenizer_decoder.texts_to_sequences(target_seqs)
target_data = pad_sequences(target_data, maxlen=maxlen_seq, padding='post')
target_data = to_categorical(target_data)
input_data.shape, target_data.shape

## Building model

In [ ]:
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional

n_words = len(tokenizer_encoder.word_index) + 1
n_tags = len(tokenizer_decoder.word_index) + 1
print(n_words, n_tags)

input = Input(shape=(maxlen_seq))
x = Embedding(input_dim=n_words, output_dim=128, input_length=maxlen_seq)(input)
x = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=0.1))(x)
y = TimeDistributed(Dense(n_tags, activation="softmax"))(x)
model = Model(input, y)
model.summary()

## Train and Evaluate Model

In [ ]:
from sklearn.model_selection import train_test_split
from keras.metrics import categorical_accuracy
from keras import backend  as K
import tensorflow as tf

def q3_acc(y_true, y_pred):
    y = tf.argmax(y_true, axis=-1)
    y_ = tf.argmax(y_pred, axis=-1)
    mask = tf.greater(y, 0)
    return K.cast(K.equal(tf.boolean_mask(y, mask), tf.boolean_mask(y_, mask)), K.floatx())

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy", q3_acc])

X_train, X_test, y_train, y_test = train_test_split(input_data, target_data, test_size=.4, random_state=0)
seq_train, seq_test, target_train, target_test = train_test_split(input_seqs, target_seqs, test_size=.4, random_state=0)

history = model.fit(X_train, y_train, batch_size=128, epochs=20, validation_data=(X_test, y_test), verbose=1)

In [ ]:
import matplotlib.pyplot as plt
# Plot training history
def viz_hist(history):
    plt.figure(figsize=(12, 6))

    # Plot training & validation loss values
    plt.subplot(1, 3, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training & validation accuracy values
    plt.subplot(1, 3, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training & validation accuracy values
    plt.subplot(1, 3, 3)
    plt.plot(history.history['q3_acc'], label='Training Q3 Accuracy')
    plt.plot(history.history['val_q3_acc'], label='Validation Q3 Accuracy')
    plt.title('Model Q3 Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
viz_hist(history)

In [ ]:
def onehot_to_seq(oh_seq, index):
    s = ''
    for o in oh_seq:
        i = np.argmax(o)
        if i != 0:
            s += index[i]
        else:
            break
    return s

def plot_results(x, y, y_):
    print("---")
    print("Input: " + str(x))
    print("Target: " + str(onehot_to_seq(y, revsere_decoder_index).upper()))
    print("Result: " + str(onehot_to_seq(y_, revsere_decoder_index).upper()))
    fig = plt.figure(figsize=(10,2))
    plt.imshow(y.T, cmap='Blues')
    plt.imshow(y_.T, cmap='Reds', alpha=.5)
    plt.yticks(range(4), [' '] + [revsere_decoder_index[i+1].upper() for i in range(3)])
    plt.show()
    
revsere_decoder_index = {value:key for key,value in tokenizer_decoder.word_index.items()}
revsere_encoder_index = {value:key for key,value in tokenizer_encoder.word_index.items()}

N=3
y_train_pred = model.predict(X_train[:N])
y_test_pred = model.predict(X_test[:N])
print('training')
for i in range(N):
    plot_results(seq_train[i], y_train[i], y_train_pred[i])
print('testing')
for i in range(N):
    plot_results(seq_test[i], y_test[i], y_test_pred[i])

## Optimizing Hyperparameter

In [ ]:
!pip install optuna

In [ ]:
def objective(trial):
    # Get the hyperparameters
    dropout = trial.suggest_float('dropout', 0.1, 0.2)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'softmax'])

    # Create the BiLSTM model
    input = Input(shape=(maxlen_seq))
    x = Embedding(input_dim=n_words, output_dim=128, input_length=maxlen_seq)(input)
    x = Bidirectional(LSTM(units=64, return_sequences=True, recurrent_dropout=dropout))(x)
    y = TimeDistributed(Dense(n_tags, activation=activation))(x)
    model = Model(input, y)
    
    # Compile the model
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy", q3_acc])


    # Train the model
    history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), verbose=1)
    #viz 
    viz_hist(history)
#     y_res = model.predict(X_test)
#     return q3_acc(y_test,y_res)

In [ ]:
import optuna
study = optuna.create_study()
study.optimize(objective, n_trials=10)
# best_dropout = study.best_params['dropout']
# best_activation = study.best_params['activation']

# ML to predict secondary structure

In [ ]:
import numpy as np                                     # linear algebra
import pandas as pd                                    # data processing, CSV file I/O (e.g. pd.read_csv)
import copy                                            #to copy list
from sklearn.model_selection import train_test_split   #to split dataset into train and test set
from sklearn.svm import SVC                            #to create svc instance
from sklearn.metrics import classification_report      #to create report for precision,recall,f1-score,accuracy
from sklearn import metrics                            #to get accuracy
from sklearn.model_selection import GridSearchCV       #to optimise the hyper-parameter
import math

## Pre processing

In [ ]:
#dataset 1
maxlen_seq = 128
input_seqs, target_seqs = df[['seq', 'sst3']][(df.len <= maxlen_seq) & (~df.has_nonstd_aa)].values.T
input_seqs, target_seqs = df[['seq', 'sst3']][(~df.has_nonstd_aa)].values.T
# input_grams = seq2ngrams(input_seqs)
print(input_seqs[0:5])

In [ ]:
inputSeqs=[]
targetSeqs=[]
for i in range(input_seqs.size):
    j=0
    while(j<len(input_seqs[i])/128):
        start = j*128
        end = start+128
        inputSeqs.append(input_seqs[i][start:end])
        targetSeqs.append(target_seqs[i][start:end])
        j+=1
        
print(len(targetSeqs))
print(len(inputSeqs))

**Looking for incomplete data**

In [ ]:
for row in range(len(targetSeqs)):
    secondary_lenth = len(targetSeqs[row])
    primary_lenth = len(inputSeqs[row])
    
    if(secondary_lenth != primary_lenth):
        print("(",row,") Secondary_Structure ->", targetSeqs[row]," Primary_Structure -> ",inputSeqs[row])
    
print(len(inputSeqs))

In [ ]:
secondary_count = 0
primary_count = 0
dataCheck = "ACEDGFIHKMLNQPSRTWVY"
index=[]
for row in range(len(targetSeqs)):
    secondary_lenth = len(targetSeqs[row])
    primary_lenth = len(inputSeqs[row])
    secondary_count = secondary_count + secondary_lenth
    primary_count = primary_count + primary_lenth
    if(secondary_lenth != primary_lenth):
        print("(",row,") Secondary_Structure ->", targetSeqs[row]," Primary_Structure -> ",inputSeqs[row])
    for col in range(len(inputSeqs[row])):
        #print("before :",inputSeqs[row][col])
        if len(inputSeqs[row])<2:
            index.append(row)
        if dataCheck.find(inputSeqs[row][col])==-1:
            #print("after :",inputSeqs[row][col])
            index.append(row)
           # print("Row : "+str(row)+"have been deleted for having unknown data")
            break
            

inputSeqs =np.delete(inputSeqs,index)
targetSeqs =np.delete(targetSeqs,index)
        
print("count of secondary structure : ",secondary_count)
print("count of primary structure : ",primary_count)
print("size of primary structure : ",len(inputSeqs))

**Orthogonal Encoding - Target Labeling**

In [ ]:
def split(sequence): 
    return [char for char in sequence]

In [ ]:
primary_split = []
secondary_split = []
for row in range(int(len(targetSeqs)/1)):
    primary_split.append(split(inputSeqs[row]))
    secondary_split.append(split(targetSeqs[row]))
    
print(len(primary_split))

The results of the split primary and secondary structure of the protein are then converted into orthogonal encoding and target labeling. A switch case snippet for each amino acid in the primary structure of a protein as follows .<br>
Secondary structure character represent ->
1. H= α-helix
2. C= Loops and irregular elements
3. E= β-strand
4. B= β-bridge
5. G= 3-helix
6. I= π-helix
7. T= Turn
8. S= Bend

In [ ]:
def orthogonal_primary(arg):
    switch = {
        'A' : np.array([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),  # 20 amino acids
        'C' : np.array([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
        'E' : np.array([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
        'D' : np.array([0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
        'G' : np.array([0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
        'F' : np.array([0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
        'I' : np.array([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]),
        'H' : np.array([0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]),
        'K' : np.array([0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]),
        'M' : np.array([0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]),
        'L' : np.array([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]),
        'N' : np.array([0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]),
        'Q' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]),
        'P' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]),
        'S' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]),
        'R' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]),
        'T' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]),
        'W' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]),
        'V' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]),
        'Y' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]),
        'X' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    }
    
    return switch.get(arg)

def orthogonal_secondary(arg):
    # H : 0, C : 1, E : 2
    # C: C, B, T, S; H: H, G, I; E: E
    switch = {
        'H' : 0,                   
        'C' : 1,
        'E' : 2,
    }
    
    return switch.get(arg)

In [ ]:
for row in range(len(primary_split)):  
    sequence = primary_split[row]
    for col in range(len(sequence)):
        #print(sequence[col])
        sequence[col] = orthogonal_primary(sequence[col])
        #print(sequence[col])

In [ ]:
for row in range(len(secondary_split)):  
    sequenceS = secondary_split[row]
    for col in range(len(sequenceS)):
        sequenceS[col] = orthogonal_secondary(sequenceS[col])

## GNN

**graph_sum2**<br>
this function take input 2 node (amino acid character's onehot key) and return the sum of 2 node .<br>
**graph_sum3**<br>
this function take input 3 node (amino acid character's onehot key) and return the sum of 3 node .

In [ ]:
def graph_sum2(seq1,seq2):
    result=[None]*len(seq1)
    for col in range(len(seq1)):
        result[col] =  seq1[col]+seq2[col]
    return result


def graph_sum3(seq1,seq2,seq3):
    result=[None]*len(seq1)
    for col in range(len(seq1)):
        result[col] =  seq1[col]+seq2[col]+seq3[col]
    return result

**Graph of primary structure**<br>
The primary structure is a linear string of character/amino acid(node) .<br>
Example : ***ABBA***<br>
In Graph Neural Network , we take each node and sum the information value of it's adjacent node. As a result we find a new value for each node which is dependable for it's adjacent nodes .<br>
the border node will use graph_sum2 function as it has only 1 adjacent node and the rest will use graph_sum3 function .

In [ ]:
graph_input = copy.deepcopy(primary_split)
for row in range(len(primary_split)):
    sequence = primary_split[row]
    graph_input[row][0]=graph_sum2(sequence[0],sequence[1])
    graph_input[row][len(sequence)-1]=graph_sum2(sequence[len(sequence)-1],sequence[len(sequence)-2])
    for col in range(1,len(sequence)-1):
        graph_input[row][col] = graph_sum3(sequence[col-1],sequence[col],sequence[col+1])
        
graph_input[0]

In [ ]:
def targetY(data_list):
    Y = []
    for i in range(len(data_list)):
        for j  in range(len(data_list[i])):
            Y.append(data_list[i][j])
    return Y

In [ ]:
y_label = targetY(secondary_split)
print(len(y_label))
print(y_label[0:5])

The data feature is formed using the window_padding_data function. This function will accept the size of the sliding window and sequence of the primary structure of the protein. In this function features will be processed such as adding padding 0 at the beginning and end and taking the features of the results of windowing so that the output data can be directly trained on the SVM model

In [ ]:
def window_padding_data(size, sequence):
    num = int(size/2)
    #print("initial :",sequence[0])
    #print("")
    zeros = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    for i in range(len(sequence)):
        for j in range(num):
            sequence[i].append(zeros)
            sequence[i].insert(0, zeros)
            #print(sequence[i])
            #print("")
            
    X = []
    temp = []

    for k in range(len(sequence)):
        #print(sequence[k])
        for l in range(len(sequence[k])-(size-1)):
            temp = sequence[k][l:l+size]
           # print(temp)
            X.append(temp)
            temp = []

    return X

In [ ]:
X = window_padding_data(11,graph_input)
print(len(X))
X[0]

In [ ]:
np.set_printoptions(threshold=np.inf)
X = np.array(X)
y_label = np.array(y_label)
X = X.reshape(len(X),11*20)
print(X[0:5])
print("X_train length :",len(X))
print("y_label length :",len(y_label))

In [ ]:

# Convert the numpy array to a Pandas DataFrame
train = pd.DataFrame(X)
train['label'] = y_label.tolist()
train.info()

In [ ]:
train['label'].value_counts()

## Training

### AutoGluon: Choosing the Model

In [ ]:
!pip install autogluon

In [ ]:
import gc
gc.collect()

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

train_data = TabularDataset(train)

label = 'label'
save_path = 'model_ag'
predictor = TabularPredictor(label=label, path=save_path, problem_type = 'multiclass', eval_metric="f1_macro", sample_weight = 'balance_weight').fit(train_data)

In [ ]:
predictor.leaderboard()

In [ ]:
predictor = TabularPredictor.load("/kaggle/working/"+save_path+'/')
predictor.info()

### Optimized Hyperparameter using Optuna

In [ ]:
import lightgbm as lgb
import sklearn
import optuna

In [ ]:

def objective(trial):
#     iris = sklearn.datasets.load_iris()
#     x, y = iris.data, iris.target
    x = X
    y = y_label
    classifier_name = trial.suggest_categorical("classifier", ["SVC", "RandomForest", "lgb"])

    if classifier_name == "SVC":
        svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
        classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
    elif classifier_name == "lgb":
        lgb_num_leaves = trial.suggest_int("lgb_num_leaves", 31, 2**16-1, log=True)
        classifier_obj = lgb.LGBMClassifier(num_leaves=lgb_num_leaves)
    else:
        rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
        classifier_obj = sklearn.ensemble.RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=10
        )

    score = sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy

# 3. Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)